In [1]:
import carla
import math
import random
import time


def main():
    client = carla.Client("localhost", 2000)
    client.set_timeout(10.0)

    world = client.get_world()
    world_map = world.get_map()

    debug = world.debug

    # Distance between waypoints in meters
    step = 2.0

    print("Generating waypoints...")
    waypoints = world_map.generate_waypoints(step)
    print(f"Got {len(waypoints)} waypoints")

    # Optional: only draw on Driving lanes
    waypoints = [wp for wp in waypoints if wp.lane_type == carla.LaneType.Driving]

    print(f"Filtered to {len(waypoints)} driving waypoints")

    # How long to keep debug drawing in the world (seconds)
    life_time = 60.0

    for wp in waypoints:
        loc = wp.transform.location
        yaw = math.radians(wp.transform.rotation.yaw)

        # Base point: center of lane
        base = carla.Location(x=loc.x, y=loc.y, z=loc.z + 0.2)

        # Direction arrow: small line in heading direction
        length = 1.0
        end = carla.Location(
            x=loc.x + length * math.cos(yaw),
            y=loc.y + length * math.sin(yaw),
            z=loc.z + 0.2,
        )

        # Color can encode lane_id or road_id if you want
        color = carla.Color(0, 255, 0)  # green

        # Draw a point at the waypoint
        debug.draw_point(
            base,
            size=0.05,
            color=color,
            life_time=life_time
        )

        # Draw a line indicating lane direction
        debug.draw_line(
            base,
            end,
            thickness=0.03,
            color=color,
            life_time=life_time
        )

    print("Waypoints drawn. Fly around with the spectator to inspect them.")
    # Keep script alive so drawings don't vanish immediately
    time.sleep(life_time + 2.0)


if __name__ == "__main__":
    main()


Generating waypoints...
Got 7092 waypoints
Filtered to 7092 driving waypoints
Waypoints drawn. Fly around with the spectator to inspect them.


KeyboardInterrupt: 

In [4]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
xodr = world.get_map().to_opendrive()

In [19]:
import carla
import math
from collections import deque
import numpy as np
import random
import time


# -----------------------------
# Core utilities
# -----------------------------

def _waypoint_key(wp: carla.Waypoint):
    """
    Key to avoid revisiting the same waypoint in BFS.
    s is the longitudinal coordinate along the road; rounded for stability.
    """
    return (wp.road_id, wp.section_id, wp.lane_id, round(wp.s, 1))


def collect_waypoints_within_radius(
    carla_map: carla.Map,
    ego_loc: carla.Location,
    radius: float = 60.0,
    step: float = 2.0,
    lane_type: carla.LaneType = carla.LaneType.Any,
):
    """
    BFS from the waypoint closest to ego_loc, following lane connections
    (forward/backward + adjacent lanes), collecting waypoints whose
    world distance from ego_loc is <= radius.
    """
    start_wp = carla_map.get_waypoint(
        ego_loc,
        project_to_road=True,
        lane_type=lane_type,
    )

    queue = deque([start_wp])
    visited = set()
    result = []

    while queue:
        wp = queue.popleft()
        key = _waypoint_key(wp)
        if key in visited:
            continue
        visited.add(key)

        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)

        # Stop expanding this branch if we've passed the radius
        if dist > radius:
            continue

        result.append(wp)

        # Neighbors along the same lane
        for nxt in wp.next(step):
            queue.append(nxt)
        for prv in wp.previous(step):
            queue.append(prv)

        # Adjacent lanes of the same type (optional)
        left = wp.get_left_lane()
        if left is not None and left.lane_type == wp.lane_type:
            queue.append(left)

        right = wp.get_right_lane()
        if right is not None and right.lane_type == wp.lane_type:
            queue.append(right)

    return result


def get_n_nearest_map_points(
    carla_map: carla.Map,
    ego_loc: carla.Location,
    radius: float = 60.0,
    n_points: int = 200,
    step: float = 2.0,
    lane_type: carla.LaneType = carla.LaneType.Driving,
):
    """
    From a CARLA map and an ego world location, return up to N nearest
    waypoint-based "map points" within a given radius.

    Returns:
        points: np.ndarray of shape (N, D), where each row is:
            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id, reserved]
        If there are fewer than N points, the rest are zero-padded.
    """
    waypoints = collect_waypoints_within_radius(
        carla_map,
        ego_loc,
        radius=radius,
        step=step,
        lane_type=lane_type,
    )

    candidates = []
    for wp in waypoints:
        loc = wp.transform.location
        yaw_rad = math.radians(wp.transform.rotation.yaw)

        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)

        half_length = step * 0.5         # rough segment half-length
        half_width = wp.lane_width * 0.5

        lane_type_code = float(wp.lane_type)  # enum -> float
        road_id = float(wp.road_id)

        candidates.append(
            (dist, loc.x, loc.y, yaw_rad, half_length, half_width, lane_type_code, road_id)
        )

    D = 8
    points = np.zeros((n_points, D), dtype=np.float32)

    if not candidates:
        return points

    # Sort by distance and keep up to N nearest
    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid, lane_t, rid) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, 0.0]

    return points


# -----------------------------
# Test code (run in your notebook)
# -----------------------------

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
carla_map = world.get_map()
debug = world.debug
random.seed(42)
print("Connected to world:", world.get_map().name)

# Choose a random waypoint as "ego"
all_wps = carla_map.generate_waypoints(distance=5.0)
ego_wp = random.choice(all_wps)
ego_loc = ego_wp.transform.location

print("Random ego at:", ego_loc)

# Get N nearest map points
RADIUS = 15
N = 5000

points = get_n_nearest_map_points(
    carla_map,
    ego_loc,
    radius=RADIUS,
    n_points=N,
    step=2.0,
    lane_type=carla.LaneType.Driving,
)

print("Points array shape:", points.shape)
print("First 5 points:\n", points[:5])

# Visualize some of them in the world (up to 50 non-zero points)
max_draw = 5000
num_drawn = 0
for row in points:
    x, y, yaw, hlen, hwidth, lane_t, rid, _ = row
    if hlen == 0 and hwidth == 0 and x == 0 and y == 0:
        continue  # skip zero-padded rows
    loc = carla.Location(x=float(x), y=float(y), z=ego_loc.z + 0.5)
    debug.draw_point(
        loc,
        size=0.1,
        color=carla.Color(0, 255, 0),
        life_time=2000.0,
    )
    num_drawn += 1
    if num_drawn >= max_draw:
        break

# Mark the ego location as well
debug.draw_point(
    ego_loc + carla.Location(z=0.7),
    size=0.15,
    color=carla.Color(255, 0, 0),
    life_time=2000.0,
)

print(f"Drew {num_drawn} map points (green) and ego (red). Fly around with the spectator to inspect.")


Connected to world: Carla/Maps/Town03
Random ego at: Location(x=2.147921, y=156.603455, z=0.000000)
Points array shape: (5000, 8)
First 5 points:
 [[2.1479211e+00 1.5660345e+02 4.7060614e+00 1.0000000e+00 1.7500000e+00
  2.0000000e+00 1.0300000e+03 0.0000000e+00]
 [2.1352663e+00 1.5460352e+02 4.7060614e+00 1.0000000e+00 1.7500000e+00
  2.0000000e+00 3.1000000e+01 0.0000000e+00]
 [2.1605759e+00 1.5860342e+02 4.7060614e+00 1.0000000e+00 1.7500000e+00
  2.0000000e+00 1.0300000e+03 0.0000000e+00]
 [5.6478505e+00 1.5658131e+02 4.7060614e+00 1.0000000e+00 1.7500000e+00
  2.0000000e+00 1.0300000e+03 0.0000000e+00]
 [2.1226115e+00 1.5260355e+02 4.7060614e+00 1.0000000e+00 1.7500000e+00
  2.0000000e+00 3.1000000e+01 0.0000000e+00]]
Drew 38 map points (green) and ego (red). Fly around with the spectator to inspect.


In [1]:
import carla
import math
from collections import deque
import numpy as np
import random
import time

# ============================================
# Core helpers
# ============================================

def _waypoint_key(wp: carla.Waypoint):
    """
    Key to avoid revisiting the same waypoint in BFS.
    s is the longitudinal coordinate along the road; rounded for stability.
    """
    return (wp.road_id, wp.section_id, wp.lane_id, round(wp.s, 1))


def collect_waypoints_within_radius(
    carla_map: carla.Map,
    ego_loc: carla.Location,
    radius: float = 60.0,
    step: float = 2.0,
    lane_type: carla.LaneType = carla.LaneType.Driving,
):
    """
    BFS from the waypoint closest to ego_loc, following lane connections
    (forward/backward + adjacent lanes), collecting waypoints whose
    world distance from ego_loc is <= radius.
    """
    start_wp = carla_map.get_waypoint(
        ego_loc,
        project_to_road=True,
        lane_type=lane_type,
    )

    queue = deque([start_wp])
    visited = set()
    result = []

    while queue:
        wp = queue.popleft()
        key = _waypoint_key(wp)
        if key in visited:
            continue
        visited.add(key)

        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)

        # Stop expanding this branch if we've passed the radius
        if dist > radius:
            continue

        result.append(wp)

        # Neighbors along the same lane
        for nxt in wp.next(step):
            queue.append(nxt)
        for prv in wp.previous(step):
            queue.append(prv)

        # Adjacent lanes of the same type (optional)
        left = wp.get_left_lane()
        if left is not None and left.lane_type == wp.lane_type:
            queue.append(left)

        right = wp.get_right_lane()
        if right is not None and right.lane_type == wp.lane_type:
            queue.append(right)

    return result


def get_n_nearest_points_with_lane_drawings(
    carla_map: carla.Map,
    ego_loc: carla.Location,
    radius: float = 60.0,
    n_points: int = 200,
    step: float = 2.0,
    lane_type: carla.LaneType = carla.LaneType.Driving,
):
    """
    From a CARLA map and an ego world location, return up to N nearest
    map-related points within a given radius.

    Logic:
      1) Find all waypoints within radius (BFS).
      2) Create "centerline" points from these waypoints.
      3) If we still haven't reached N, create lane-drawing points
         (left/right lane boundary points) for those waypoints.
      4) Sort everything by distance to ego and keep up to N points.

    Returns:
        points: np.ndarray of shape (N, D), where each row is:

            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id,
             geom_type_code, lane_mark_type_code]

        geom_type_code:
            0 = lane center
            1 = left lane marking
            2 = right lane marking

        lane_mark_type_code:
            0 for centers (no marking)
            otherwise numeric code from CARLA's LaneMarkingType (cast to float)

        If there are fewer than N points, the rest are zero-padded.
    """
    waypoints = collect_waypoints_within_radius(
        carla_map,
        ego_loc,
        radius=radius,
        step=step,
        lane_type=lane_type,
    )

    candidates = []

    # --------------------------------
    # 1) Centerline points (geom_type = 0)
    # --------------------------------
    for wp in waypoints:
        loc = wp.transform.location
        yaw_rad = math.radians(wp.transform.rotation.yaw)

        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)

        half_length = step * 0.5
        half_width = wp.lane_width * 0.5

        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lane_mark_type_code = 0.0  # no specific marking, it's center

        candidates.append(
            (
                dist,                 # for sorting
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code,
                lane_mark_type_code,
            )
        )

    # --------------------------------
    # 2) Lane drawing points (left/right markings)
    #    Only if we still might need more points.
    # --------------------------------
    # We don't know yet if we exceed n_points, but it's fine:
    # we'll sort + truncate later.
    for wp in waypoints:
        loc = wp.transform.location
        center_x = loc.x
        center_y = loc.y
        yaw_rad = math.radians(wp.transform.rotation.yaw)

        # Unit normal to the left of the lane direction
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)

        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # LEFT marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = center_x + nx * half_width
            by = center_y + ny * half_width

            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)

            geom_type_code = 1.0
            # numeric code for lane marking type; adapt if needed
            lane_mark_type_code = float(int(lm_left.type))

            candidates.append(
                (
                    dist,
                    bx, by,
                    yaw_rad,
                    half_length, 0.1,  # line is very thin
                    lane_type_code, road_id,
                    geom_type_code,
                    lane_mark_type_code,
                )
            )

        # RIGHT marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = center_x - nx * half_width
            by = center_y - ny * half_width

            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)

            geom_type_code = 2.0
            lane_mark_type_code = float(int(lm_right.type))

            candidates.append(
                (
                    dist,
                    bx, by,
                    yaw_rad,
                    half_length, 0.1,
                    lane_type_code, road_id,
                    geom_type_code,
                    lane_mark_type_code,
                )
            )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)

    if not candidates:
        return points

    # Sort by distance to ego and keep up to N
    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [
            x, y,
            yaw,
            hlen, hwid,
            lane_t, rid,
            geom_t, lm_type,
        ]

    return points


# ============================================
# Test code (run directly in your notebook)
# ============================================

# Connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
carla_map = world.get_map()
debug = world.debug

print("Connected to world:", world.get_map().name)

# Choose a random waypoint as "ego"
all_wps = carla_map.generate_waypoints(distance=10.0)
ego_wp = random.choice(all_wps)
ego_loc = ego_wp.transform.location

print("Random ego at:", ego_loc)

# Parameters
RADIUS = 60.0
N = 1000

points = get_n_nearest_points_with_lane_drawings(
    carla_map,
    ego_loc,
    radius=RADIUS,
    n_points=N,
    step=2.0,
    lane_type=carla.LaneType.Driving,
)

print("Points array shape:", points.shape)
print("First 5 points:\n", points[:5])

# Visualize some of them in the world
max_draw = 1000
num_drawn = 0

for row in points:
    x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type = row
    if x == 0 and y == 0 and hlen == 0 and hwid == 0:
        continue  # skip zero-padded rows

    z = ego_loc.z + 0.5
    loc = carla.Location(x=float(x), y=float(y), z=z)

    # Color by geom_type: center = green, left mark = blue, right mark = yellow
    if int(geom_t) == 0:
        color = carla.Color(0, 255, 0)      # center
    elif int(geom_t) == 1:
        color = carla.Color(0, 0, 255)      # left marking
    else:
        color = carla.Color(255, 255, 0)    # right marking

    debug.draw_point(
        loc,
        size=0.08,
        color=color,
        life_time=2000.0,
    )

    num_drawn += 1
    if num_drawn >= max_draw:
        break

# Draw ego in red
debug.draw_point(
    ego_loc + carla.Location(z=0.7),
    size=0.15,
    color=carla.Color(255, 0, 0),
    life_time=2000.0,
)

print(f"Drew {num_drawn} map points (centers + lane drawings) and ego (red).")


Connected to world: Carla/Maps/Town10HD_Opt
Random ego at: Location(x=-52.158344, y=52.655090, z=0.000000)
Points array shape: (1000, 9)
First 5 points:
 [[-52.158344   52.655087    1.5679823   1.          1.75        2.
  933.          0.          0.       ]
 [-52.158077   52.74875     1.5679823   1.          1.75        2.
  933.          0.          0.       ]
 [-52.158707   52.525204    1.5679823   1.          1.75        2.
  933.          0.          0.       ]
 [-52.157703   52.881954    1.5679823   1.          1.75        2.
  933.          0.          0.       ]
 [-52.159027   52.41203     1.5679823   1.          1.75        2.
  933.          0.          0.       ]]
Drew 1000 map points (centers + lane drawings) and ego (red).


In [9]:

# Parameters
RADIUS = 60.0
N = 1000

points = get_n_nearest_points_with_lane_drawings(
    carla_map,
    ego_loc,
    radius=RADIUS,
    n_points=N,
    step=2.0,
    lane_type=carla.LaneType.Driving,
)

print("Points array shape:", points.shape)
print("First 5 points:\n", points[:5])

# Visualize some of them in the world
max_draw = 1000
num_drawn = 0

for row in points:
    x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type = row
    if x == 0 and y == 0 and hlen == 0 and hwid == 0:
        continue  # skip zero-padded rows

    z = ego_loc.z + 0.5
    loc = carla.Location(x=float(x), y=float(y), z=z)

    # Color by geom_type: center = green, left mark = blue, right mark = yellow
    if int(geom_t) == 0:
        color = carla.Color(0, 255, 0)      # center
    elif int(geom_t) == 1:
        color = carla.Color(0, 0, 255)      # left marking
    else:
        color = carla.Color(255, 255, 0)    # right marking

    debug.draw_point(
        loc,
        size=0.08,
        color=color,
        life_time=2000.0,
    )

    num_drawn += 1
    if num_drawn >= max_draw:
        break

# Draw ego in red
debug.draw_point(
    ego_loc + carla.Location(z=0.7),
    size=0.15,
    color=carla.Color(255, 0, 0),
    life_time=2000.0,
)

print(f"Drew {num_drawn} map points (centers + lane drawings) and ego (red).")


In [3]:
import carla
import math

def draw_local_map_points(
    world,
    geo_location,
    radius=60.0,
    n_points=200,
    step=2.0,
    lane_type=carla.LaneType.Driving,
    life_time=20.0,
    max_draw=None,
):
    """
    1) Find the closest waypoint to the given geo_location.
    2) Use that waypoint's location as ego.
    3) Get up to n_points map points (centers + lane drawings) within radius.
    4) Draw them in the world.

    Returns:
        points: np.ndarray of shape (n_points, 9)
        ego_loc: carla.Location used as ego
    """
    if max_draw is None:
        max_draw = n_points

    carla_map = world.get_map()
    debug = world.debug

    # 1) Closest waypoint to given geo_location
    ego_wp = carla_map.get_waypoint(
        geo_location,
        project_to_road=True,
        lane_type=lane_type,
    )
    ego_loc = ego_wp.transform.location

    # 2) Get map points around ego
    points = get_n_nearest_points_with_lane_drawings(
        carla_map,
        ego_loc,
        radius=radius,
        n_points=n_points,
        step=step,
        lane_type=lane_type,
    )

    # 3) Draw them
    num_drawn = 0
    for row in points:
        x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type = row
        if x == 0 and y == 0 and hlen == 0 and hwid == 0:
            continue  # skip zero-padded rows

        z = ego_loc.z + 0.5
        loc = carla.Location(x=float(x), y=float(y), z=z)

        # Color by geom_type: center = green, left mark = blue, right mark = yellow
        if int(geom_t) == 0:
            color = carla.Color(0, 255, 0)      # center
        elif int(geom_t) == 1:
            color = carla.Color(0, 0, 255)      # left marking
        else:
            color = carla.Color(255, 255, 0)    # right marking

        debug.draw_point(
            loc,
            size=0.08,
            color=color,
            life_time=life_time,
        )

        num_drawn += 1
        if num_drawn >= max_draw:
            break

    # Draw ego in red
    debug.draw_point(
        ego_loc + carla.Location(z=0.7),
        size=0.15,
        color=carla.Color(255, 0, 0),
        life_time=life_time,
    )

    print(f"Drew {num_drawn} map points (centers + lane drawings) and ego (red).")
    return points, ego_loc


In [27]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

carla_map = world.get_map()
all_wps = carla_map.generate_waypoints(distance=10.0)
random_wp = random.choice(all_wps)
geo_location = carla.Location(x=-28.8,y=130.0) 
points, ego_loc = draw_local_map_points(
    world,
    geo_location=geo_location,
    radius=22.0,
    n_points=10000,
    step=1.0,
    life_time=10.0,
)

print("Points shape:", points.shape)
print("Ego used:", ego_loc)

Drew 545 map points (centers + lane drawings) and ego (red).
Points shape: (10000, 9)
Ego used: Location(x=-28.791756, y=130.033875, z=0.000000)


# Brand new No BFS approach


In [29]:
import carla
import math
import numpy as np

def precompute_driving_waypoints(carla_map, step=2.0):
    """
    Precompute all driving waypoints for the map at given resolution.
    Call this once and reuse the result.
    """
    all_wps = carla_map.generate_waypoints(distance=step)
    driving_wps = [wp for wp in all_wps if wp.lane_type == carla.LaneType.Driving]
    return driving_wps

def get_local_points_from_precomputed(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
):
    """
    No BFS. Just:
      - filter precomputed waypoints by radius
      - add center points
      - add lane marking points
      - sort by distance and keep up to n_points
    """
    candidates = []

    # 1) Center points
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist > radius:
            continue

        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane drawings (left/right markings) for the same wps
    for wp in driving_wps:
        loc = wp.transform.location
        dx0 = loc.x - ego_loc.x
        dy0 = loc.y - ego_loc.y
        base_dist = math.hypot(dx0, dy0)
        if base_dist > radius:
            continue

        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # left marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # right marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    return points


In [30]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

# Precompute once
driving_wps = precompute_driving_waypoints(carla_map, step=2.0)

# Pick a fixed ego point
all_wps = carla_map.generate_waypoints(10.0)
ego_wp = all_wps[0]
ego_loc = carla.Location(x=-44.0,y=129.7)

for r in [21.0, 22.0, 23.0]:
    pts = get_local_points_from_precomputed(
        driving_wps,
        ego_loc,
        radius=r,
        n_points=5000,
        step=2.0,
    )
    valid = (pts[:,0] != 0) | (pts[:,1] != 0)
    print(f"radius {r}: {valid.sum()} points")


radius 21.0: 323 points
radius 22.0: 342 points
radius 23.0: 362 points


# Complete radius sampling code

In [1]:
import carla
import math
import numpy as np
import random
import time

In [2]:
def precompute_driving_waypoints(carla_map, step=2.0):
    """
    Precompute all driving waypoints for the map at given resolution.
    Call this once and reuse the result.
    """
    all_wps = carla_map.generate_waypoints(distance=step)
    driving_wps = [wp for wp in all_wps if wp.lane_type == carla.LaneType.Driving]
    print(f"Precomputed {len(driving_wps)} driving waypoints (step={step})")
    return driving_wps

In [3]:
def precompute_landmarks(carla_map):
    """
    Precompute all landmarks from the map.
    """
    all_landmarks = carla_map.get_all_landmarks()
    print(f"Precomputed {len(all_landmarks)} landmarks")
    return all_landmarks


In [4]:
def get_local_points_from_precomputed_knearest(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
    all_landmarks=None,
):
    """
    Using precomputed driving waypoints:
      - Find waypoints whose *center* is within radius of ego_loc.
      - Build lane center points for them.
      - Build lane marking points (left/right) from them.
      - Keep only those boundary points whose own position is within radius.
      - Sort all candidates by distance and keep up to n_points.

    Returns:
        points: np.ndarray of shape (n_points, 9), rows:

            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id,
             geom_type_code, lane_mark_type_code]

        geom_type_code:
            0 = lane center
            1 = left lane marking
            2 = right lane marking

        lane_mark_type_code:
            0 for centers
            numeric code (float) for lane marking types for boundary points
    """
    candidates = []

    # First: select only waypoints whose center is within radius
    wps_near = []
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist <= radius:
            wps_near.append((wp, loc, dist))

    # 1) Lane center points
    for wp, loc, dist in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane marking points (left/right), built from the same nearby waypoints
    for wp, loc, _ in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # LEFT marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,      # very thin strip
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # RIGHT marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )
    # 3) Landmarks (optional)
    if all_landmarks is not None:
        for lm in all_landmarks:
            loc = lm.transform.location
            dx = loc.x - ego_loc.x
            dy = loc.y - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist > radius:
                continue

            yaw_rad = math.radians(lm.transform.rotation.yaw)
            # Rough size guesses; you can tune these per lm.type if you like
            half_length = 1.0
            half_width = 1.0

            geom_type_code = 3.0  # "landmark"
            # landmark.type is often a numeric string like "1000001"
            try:
                subtype = float(int(lm.type))
            except (ValueError, TypeError):
                subtype = 0.0

            # Store landmark id in the "road_id" slot
            lm_id = float(lm.id)

            candidates.append(
                (
                    dist,
                    loc.x, loc.y,
                    yaw_rad,
                    half_length, half_width,
                    0.0, lm_id,        # lane_type=0 for landmarks
                    geom_type_code, subtype,
                )
            )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    # Sort by distance and keep up to n_points
    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    return points


In [5]:
def get_local_points_from_precomputed_downsample(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
    all_landmarks=None,
):
    """
    Using precomputed driving waypoints:
      - Find waypoints whose *center* is within radius of ego_loc.
      - Build lane center points for them.
      - Build lane marking points (left/right) from them.
      - Keep only those boundary points whose own position is within radius.
      - Sort all candidates by distance.
      - If there are more than n_points, UNIFORMLY DOWNSAMPLE across distance.

    Returns:
        points: np.ndarray of shape (n_points, 9), rows:

            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id,
             geom_type_code, lane_mark_type_code]
    """
    candidates = []

    # First: waypoints whose center is within radius
    wps_near = []
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist <= radius:
            wps_near.append((wp, loc, dist))

    # 1) Lane center points
    for wp, loc, dist in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane marking points (left/right)
    for wp, loc, _ in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # LEFT marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # RIGHT marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    # Sort by distance
    candidates.sort(key=lambda t: t[0])
    total = len(candidates)

    # Uniform downsampling across distance axis
    if total <= n_points:
        selected = candidates
    else:
        # indices spaced evenly from closest to farthest
        idxs = np.linspace(0, total - 1, n_points, dtype=int)
        selected = [candidates[i] for i in idxs]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    # If total < n_points, remaining rows stay zero-padded
    return points


In [6]:
def draw_local_map_points_precomputed(
    world,
    driving_wps,
    all_landmarks,
    geo_location,
    radius=60.0,
    n_points=200,
    step=2.0,
    life_time=20.0,
    max_draw=None,
):
    """
    Same as before, but also draws landmarks (geom_type 3) in a distinct color.
    """
    if max_draw is None:
        max_draw = n_points

    carla_map = world.get_map()
    debug = world.debug

    ego_wp = carla_map.get_waypoint(
        geo_location,
        project_to_road=True,
        lane_type=carla.LaneType.Driving,
    )
    ego_loc = ego_wp.transform.location

    points = get_local_points_from_precomputed_knearest(
        driving_wps,
        ego_loc,
        radius=radius,
        n_points=n_points,
        step=step,
        all_landmarks=all_landmarks,
    )

    num_drawn = 0
    for row in points:
        x, y, yaw, hlen, hwid, lane_t, rid, geom_t, subtype = row
        if x == 0 and y == 0 and hlen == 0 and hwid == 0:
            continue

        loc = carla.Location(x=float(x), y=float(y), z=ego_loc.z + 0.5)

        if int(geom_t) == 0:
            color = carla.Color(0, 255, 0)       # lane center
        elif int(geom_t) == 1:
            color = carla.Color(0, 0, 255)       # left marking
        elif int(geom_t) == 2:
            color = carla.Color(255, 255, 0)     # right marking
        else:
            color = carla.Color(255, 0, 0)       # landmarks (stop sign, etc.)

        debug.draw_point(
            loc,
            size=0.08,
            color=color,
            life_time=life_time,
        )

        num_drawn += 1
        if num_drawn >= max_draw:
            break

    debug.draw_point(
        ego_loc + carla.Location(z=0.7),
        size=0.15,
        color=carla.Color(0, 255, 255),
        life_time=life_time,
    )

    print(f"Drew {num_drawn} map points (centers + lane drawings + landmarks) and ego.")
    return points, ego_loc


In [8]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

STEP = 2.0
driving_wps = precompute_driving_waypoints(carla_map, step=STEP)
all_landmarks = precompute_landmarks(carla_map)

# pick some geo location (e.g., from ego vehicle)
all_wps_for_test = carla_map.generate_waypoints(distance=10.0)
ego_wp = random.choice(all_wps_for_test)
#geo_location = ego_wp.transform.location
ego_location = carla.Location(x=58.6, y=65.3)

RADIUS = 60.0
N_POINTS = 1000

points, ego_loc = draw_local_map_points_precomputed(
    world,
    driving_wps,
    all_landmarks,
    geo_location=ego_location,
    radius=RADIUS,
    n_points=N_POINTS,
    step=STEP,
    life_time=2000.0,
    max_draw=1000,
)

print("Points shape:", points.shape)
print("First 5 points:\n", points[:5])


Precomputed 3066 driving waypoints (step=2.0)
Precomputed 68 landmarks
Drew 1000 map points (centers + lane drawings + landmarks) and ego.
Points shape: (1000, 9)
First 5 points:
 [[ 5.8362270e+01  6.6323174e+01 -3.1403139e+00  1.0000000e+00
   1.0000000e+00  0.0000000e+00  1.0720000e+03  3.0000000e+00
   2.0600000e+02]
 [ 5.8052273e+01  6.6322777e+01 -3.1403139e+00  1.0000000e+00
   1.7500000e+00  2.0000000e+00  1.5100000e+02  0.0000000e+00
   0.0000000e+00]
 [ 5.8052273e+01  6.6322777e+01 -3.1403139e+00  1.0000000e+00
   1.7500000e+00  2.0000000e+00  1.6000000e+02  0.0000000e+00
   0.0000000e+00]
 [ 5.9982273e+01  6.6325249e+01 -3.1403139e+00  1.0000000e+00
   1.7500000e+00  2.0000000e+00  1.1000000e+01  0.0000000e+00
   0.0000000e+00]
 [ 5.8050034e+01  6.8072777e+01 -3.1403139e+00  1.0000000e+00
   1.0000000e-01  2.0000000e+00  1.5100000e+02  1.0000000e+00
   3.0000000e+00]]


### Debug: visualize all landmarks

In [39]:
import carla

# Connect (adjust host/port if needed)
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
carla_map = world.get_map()
debug = world.debug

landmarks = carla_map.get_all_landmarks()
print(f"Found {len(landmarks)} landmarks")

LIFE_TIME = 200.0

for lm in landmarks:
    loc = lm.transform.location

    # Big fat point
    debug.draw_point(
        loc + carla.Location(z=1.5),
        size=0.3,
        color=carla.Color(255, 0, 255),  # bright magenta
        life_time=LIFE_TIME,
    )

    # Label with id + type
    debug.draw_string(
        loc + carla.Location(z=2.0),
        text=f"{lm.id} | {lm.type}",
        draw_shadow=True,
        color=carla.Color(255, 255, 255),
        life_time=LIFE_TIME,
    )

print("Landmarks drawn for 200 seconds. Fly around with the spectator and look for magenta dots + labels.")


Found 68 landmarks
Landmarks drawn for 200 seconds. Fly around with the spectator and look for magenta dots + labels.


In [38]:
import carla
from collections import Counter

client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

landmarks = carla_map.get_all_landmarks()
print("Total landmarks:", len(landmarks))

counter = Counter((lm.type, lm.name) for lm in landmarks)
for (lm_type, lm_name), count in counter.most_common():
    print(f"{count:3d} x type={lm_type!r}, name={lm_name!r}")


Total landmarks: 68
 56 x type='1000001', name='Signal_3Light_Post01'
  6 x type='206', name='Sign_Stop'
  3 x type='206', name='Stencil_STOP'
  2 x type='1000001', name=''
  1 x type='205', name='Sign_Yield'
